# Lab 03 - Topic Modelling
In this lab we will look into building topic models, but will also examine dimensionality reduction and other relevant subjects.

## Latent Semantic Analysis (LSA)
Based on: [Text Mining 101: A Stepwise Introduction to Topic Modeling using Latent Semantic Analysis (using Python)](https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/)

### Data reading and inspection
Let’s load the required libraries

In [13]:
import pip
pip.main(['install','seaborn'])
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)


We will use the ’20 Newsgroup’ dataset from sklearn.

OPTIONALY: You can also download the dataset [here](https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups), if you want to look at it

In [14]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

Let's look at the 20 labels from the dataset

In [15]:
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Data Preprocessing
To start with, we will try to clean our text data as much as possible.

### Challenge 01

In [16]:
news_df = pd.DataFrame({'document':documents})

# TODO: Remove everything except alphabets
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# TODO: Remove short words (words with len < 4) from `clean_doc` column.
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# TODO: Make all text lowercase
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/tmp/ipykernel_1002/1191655101.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")


Tokenise and remove the stop-words... eventually we will stitch the text back together

### Challenge 02

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# TODO: Tokenization
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# TODO: Remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [el for el in x if el not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

### TFIDF Document-Term Matrix
This is the first step towards topic modeling. We will use sklearn’s TfidfVectorizer to create a document-term matrix with 1,000 terms.

### Challenge 03

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

# TODO: Use fit_transform func to transform `clean_doc` with TfidfVectorizer
X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # check shape of the document-term matrix

(11314, 1000)

We could have used all the terms to create this matrix but that would need quite a lot of computation time and resources. Hence, we have restricted the number of features to 1,000. If you have the computational power, I suggest trying out all the terms.

### Topic Modeling
The next step is to represent each and every term and document as a vector. We will use the document-term matrix and decompose it into multiple matrices. We will use sklearn’s TruncatedSVD to perform the task of matrix decomposition.

Since the data comes from 20 different newsgroups, let’s try to have 20 topics for our text data. The number of topics can be specified by using the n_components parameter.

In [20]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

# TODO: Fit svd_model on X
svd_model.fit(X)

len(svd_model.components_)

20

The components of svd_model are our topics, and we can access them using svd_model.components_. Finally, let’s print a few most important words in each of the 20 topics and see how our model has done.

In [21]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0]," ")

Topic 0: 
like  
know  
people  
think  
good  
time  
thanks  
Topic 1: 
thanks  
windows  
card  
drive  
mail  
file  
advance  
Topic 2: 
game  
team  
year  
games  
season  
players  
good  
Topic 3: 
drive  
scsi  
disk  
hard  
card  
drives  
problem  
Topic 4: 
windows  
file  
window  
files  
program  
using  
problem  
Topic 5: 
government  
chip  
mail  
space  
information  
encryption  
data  
Topic 6: 
like  
bike  
know  
chip  
sounds  
looks  
look  
Topic 7: 
card  
sale  
video  
offer  
monitor  
price  
jesus  
Topic 8: 
know  
card  
chip  
video  
government  
people  
clipper  
Topic 9: 
good  
know  
time  
bike  
jesus  
problem  
work  
Topic 10: 
think  
chip  
good  
thanks  
clipper  
need  
encryption  
Topic 11: 
thanks  
right  
problem  
good  
bike  
time  
window  
Topic 12: 
good  
people  
windows  
know  
file  
sale  
files  
Topic 13: 
space  
think  
know  
nasa  
problem  
year  
israel  
Topic 14: 
space  
good  
card  
people  
time  
nas

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Topics Visualization
To find out how distinct our topics are, we should visualize them. Of course, we cannot visualize more than 3 dimensions, but there are techniques like t-SNE which can help us visualize high dimensional data into lower dimensions. Here we will use a relatively new technique called UMAP (Uniform Manifold Approximation and Projection).

In [25]:
# pip install umap-learn
pip.main(['install','umap-learn'])
import umap.umap_ as umap

X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(14,10))
plt.scatter(embedding[:, 0], embedding[:, 1], 
            c = dataset.target,
            s = 10, # size
            edgecolor='none')
plt.show()

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached umap_learn-0.5.2-py3-none-any.whl
  Using cached pynndescent-0.5.6-py3-none-any.whl
  Using cached numba-0.55.1-1-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.3 MB)
  Using cached llvmlite-0.38.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached numpy-1.21.5-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.2
    Uninstalling numpy-1.22.2:
      Successfully uninstalled numpy-1.22.2


ImportError: Numba needs NumPy 1.21 or less